# Network Topology
## UC6: Digital Twin for DER
![Network Topology](../network_model/topology_UC6_meters_v9.png)

# Import the required dependencies

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import dpsimpy
from dpsim import matpower

## Convert MATPOWER format to DPsim format

In [ ]:
# Reader(mpc_file_path): mpc_file_path is relative to the Notebook dir
mpc_reader = matpower.Reader('../network_model/load_flow_pilot_district_mpc_struct.mat', 'mpc')
system = mpc_reader.load_mpc()

In [ ]:
# Plot DPsim model
system

# Read injection data

In [ ]:
import scipy.io
injections_mat = '../network_model/load_flow_pilot_injections_data.mat'
injections_raw = scipy.io.loadmat(injections_mat)
injections_data = pd.DataFrame(injections_raw['data'])
injections_data.columns = ["Pinj", "Qinj"]
injections_data=injections_data*1e6
injections_data

## Set up and run DPsim simulation

In [ ]:
import cmath

sim_name = 'pilot_uc6'
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
logger = dpsimpy.Logger(sim_name)

for node in system.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)
    
# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.info)
sim.set_system(system)
sim.set_time_step(1)
sim.set_final_time(1)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.NRP)
sim.do_init_from_nodes_and_terminals(False)

# In bus 3
sim.get_idobj_attr('load2', 'P').set(injections_data['Pinj'][2])
sim.get_idobj_attr('load2', 'Q').set(injections_data['Qinj'][2])

sim.get_idobj_attr('load3', 'P').set(injections_data['Pinj'][3])
sim.get_idobj_attr('load3', 'Q').set(injections_data['Qinj'][3])

sim.get_idobj_attr('load4', 'P').set(injections_data['Pinj'][4])
sim.get_idobj_attr('load4', 'Q').set(injections_data['Qinj'][4])

sim.add_logger(logger)
sim.run()


# Read and display DPsim simulation results

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts

log_file = 'logs/' + sim_name +'/' + sim_name + '.csv'
print(log_file)
ts_dpsimpy = rt.read_timeseries_dpsim(log_file)
results = pd.DataFrame(columns=['Bus', 'V_mag(pu)', 'V_angle(deg)', 'P(MW)', 'Q (MVAr)'])

for i in range(len(system.nodes)):
    node = system.nodes[i].name()
    node_baseV = mpc_reader.mpc_bus_data.loc[mpc_reader.mpc_bus_data['bus_i'] ==  int(node[1:]), 'baseKV'].iloc[0] * 1e3
    w_mw = 1e-6
    results.loc[i] = ([node] + [np.absolute(ts_dpsimpy[node + '.V'].values[-1]) / node_baseV]
        + [np.degrees(np.angle(ts_dpsimpy[node + '.V'].values[-1]))] 
        + [w_mw * np.real(ts_dpsimpy[node + '.S'].values[-1])] 
        + [w_mw * np.imag(ts_dpsimpy[node + '.S'].values[-1])])
    
results

# Validation: read and display MATPOWER results

In [ ]:
input_mat = '../validation/load_flow_pilot_district_power_flow_results.mat'
data = scipy.io.loadmat(input_mat)

In [ ]:
bus_data = pd.DataFrame(data['results'][0][0]['bus'])
bus_data.columns = ["bus_i", "type", "Pd", "Qd", "Gs", "Bs", "area", "Vm", "Va", "baseKV", "zone", "Vmax", "Vmin"]

bus_data

# Compare DPsim & MATPOWER results

In [ ]:
# assert will trigger an error if the condition is not met
try:
    assert all(bus_data['Vm'] - results['V_mag(pu)'] < 0.001)
    assert all(bus_data['Va'] - results['V_angle(deg)'] < 0.05)
    print ("All test were succesfull")
except:
    print ("Some test results are incorrect")